In [ ]:
from src.component.data_info import *
from datetime import datetime, timedelta

In [1]:
# specify the date we want to fetch our data
from datetime import datetime, timedelta

import pandas as pd
#specify the current time
current_date = pd.to_datetime(datetime.utcnow(),utc=True).floor('h')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=90)#it should only fetch for the last hour but because we dont want the pipline to break in case this data or the provois our was not availble so thats why we added for the entier month
fetch_data_from = pd.Timestamp(year=fetch_data_from.year, month=fetch_data_from.month, day=1,tz='UTC')
print(fetch_data_to, fetch_data_from )

current_date=Timestamp('2025-03-08 15:00:00+0000', tz='UTC')
2025-03-08 15:00:00+00:00 2024-12-01 00:00:00+00:00


C:\Users\Hadeel\AppData\Local\Temp\ipykernel_28716\3240295609.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_date = pd.to_datetime(datetime.utcnow(),utc=True).floor('h')


In [2]:
#fetch batch of data from the wherehous

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # download 2 files from website
    demand_values = load_full_data(from_date_,to_date_)
    demand_values = demand_values[(demand_values.date >= from_date_) & (demand_values.date <= to_date_)]


    # shift the data to pretend this is recent data
    demand_values['date'] += timedelta(days=7*52)

    demand_values.sort_values(by=['sub_region_code', 'date'], inplace=True)

    return demand_values

In [ ]:
data=fetch_batch_raw_data(fetch_data_from,fetch_data_to)

from_date=Timestamp('2024-12-01 00:00:00+0000', tz='UTC'), to_date_=Timestamp('2024-03-09 15:00:00+0000', tz='UTC')


NameError: name 'load_full_data' is not defined

In [ ]:
# string to datetime
data['date'] = pd.to_datetime(data['date'], utc=True)

# add column with Unix epoch milliseconds
data['seconds'] = data['date'].astype('int64') // 10**6

In [ ]:
data.isna().sum()

date               0
sub_region_code    0
demand             0
temperature_2m     0
seconds            0
dtype: int64

In [ ]:
data.tail()

,date,sub_region_code,demand,temperature_2m,seconds
26234,2025-02-04 10:00:00+00:00,10,1919,9.5085,1738663200000
26223,2025-02-04 11:00:00+00:00,10,2047,10.2585,1738666800000
26212,2025-02-04 12:00:00+00:00,10,2247,10.3585,1738670400000
26201,2025-02-04 13:00:00+00:00,10,2338,10.3585,1738674000000
26190,2025-02-04 14:00:00+00:00,10,2243,10.3585,1738677600000


In [ ]:
import hopsworks
from src.component import feature_group_config

In [ ]:
hopsworks.__version__

'4.1.5'

In [ ]:
# project = hopsworks.login(
#     project=feature_group_config.HOPSWORKS_PROJECT_NAME,
#     api_key_value=feature_group_config.HOPSWORKS_API_KEY
# )
# feature_store = project.get_feature_store()

# FEATURE_GROUP_NAME = 'electricity_demand_feature_group2'
# FEATURE_GROUP_VERSION = 3

# feature_group = feature_store.get_or_create_feature_group(
#     name=FEATURE_GROUP_NAME,
#     version=FEATURE_GROUP_VERSION,
#     description="Time-series data at hourly frequency",
#     primary_key = ['sub_region_code', 'date'],
#     event_time='date',
# )

In [ ]:
#save predictions to the feature store
from src.component.feature_store_api import get_feature_store
import src.component.feature_group_config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=3,
    description="Time-series data at hourly frequency",
    primary_key = ['sub_region_code', 'date'],
    event_time='date',
)



2025-02-04 14:15:01,290 INFO: Initializing external client
2025-02-04 14:15:01,291 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-04 14:15:02,429 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212591


In [ ]:
feature_group.insert(data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 26224/26224 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: electricity_demand_feature_group2_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1212591/jobs/named/electricity_demand_feature_group2_3_offline_fg_materialization/executions


(Job('electricity_demand_feature_group2_3_offline_fg_materialization', 'SPARK'),
 None)

: 